In [2]:

# https://github.com/pytorch/examples/blob/master/mnist/main.py
#*******************************************************************************
#Author: Seungmin.Jeong
#Purpose: Pytorch code for CNN accelerator using Verilog hdl
#Revision History: 2023.03.03
#*******************************************************************************
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np

batch_size = 64
train_dataset = datasets.MNIST(root='./data/',
                                train=True,
                                transform=transforms.ToTensor(),
                                download=True)
test_dataset = datasets.MNIST(root='./data/',
                                train=False,
                                transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=batch_size,
                                            shuffle=False)

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1_out_np = np.zeros((1, 4, 24, 24))
        self.mp1_out_np = np.zeros((1, 4, 12, 12))
        self.conv2_out_np = np.zeros((1, 12, 8, 8))
        self.mp2_out_np = np.zeros((1, 12, 4, 4))
        self.fc_in_np = np.zeros((1, 192))
        self.fc_out_np = np.zeros((1, 10))
        
        self.conv1 = nn.Conv2d(1, 4, kernel_size=5, bias=False) 
        self.conv2 = nn.Conv2d(4, 12, kernel_size=5, bias=False)
        self.mp = nn.MaxPool2d(2)
        self.fc_1 = nn.Linear(192, 10)
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.conv1(x)
        self.conv1_out_np = x.detach().numpy()
        x = F.relu(self.mp(x))
        self.mp1_out_np = x.detach().numpy()
        x = self.conv2(x)
        self.conv2_out_np = x.detach().numpy()
        x = F.relu(self.mp(x))
        self.mp2_out_np = x.detach().numpy()
        x = x.view(in_size, -1) 
        self.fc_in_np = x.detach().numpy()
        x = self.fc_1(x)
        self.fc_out_np = x.detach().numpy()
        return F.log_softmax(x)
     
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
def test():
    model.eval()
    test_loss = 0
    correct = 0
    
    
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1] 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
for epoch in range(1, 10):
    train(epoch)
    test()

C:\Users\user\AppData\Local\Temp\ipykernel_13584\3042768698.py:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308738
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.294656
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.294189
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.279483
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.281062
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.279157
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.273070
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.217522
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.212403
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.192269
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.151723
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.083134
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.996767
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.860975
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.586429
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.519836
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.288030
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.030643
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.780471
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.748837
Train Epoch: 1 [12800/60000 (

C:\Users\user\AppData\Local\Temp\ipykernel_13584\3042768698.py:94: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Average loss: 0.2693, Accuracy: 9162/10000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.413729
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.367470
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.228793
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.200536
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.318466
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.264222
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.106608
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.191345
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.209198
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.144285
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.109201
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.121232
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.212693
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.400109
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.242599
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.248527
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.151455
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.347893
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.104798
Train Epoch: 2 [12

In [ ]:
from PIL import Image

from fxpmath import Fxp


def sample_test():
    model.eval()
    test_loss = 0
    correct = 0

    target = Variable(torch.tensor([0]))
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (1).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_1.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_1.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (2).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_2.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_2.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (3).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_3.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_3.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (4).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_4.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_4.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (5).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_5.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_5.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (6).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_6.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_6.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (7).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_7.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_7.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (8).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_8.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_8.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (9).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_9.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_9.txt', np_img_re0, fmt='%d',delimiter = " ")
    img0 = Image.open("C:\\Users\\user\\Desktop\\MNIST_test_image\\9\\img9 (10).jpg", "r")
    np_img0 = np.array(img0)
    np_img_re0 = np.reshape(np_img0,(28,28))
    print(np_img_re0)
    np.savetxt('HW_test_num9_10.mem', np_img_re0, fmt='%1.2X',delimiter = " ")
    np.savetxt('SW_test_num9_10.txt', np_img_re0, fmt='%d',delimiter = " ")
    data = Variable(torch.tensor((np_img_re0 / 255), dtype = torch.float32))
    
    output = model(data)
    test_loss += F.nll_loss(output, target, reduction='sum').item()
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    print(correct)
    
sample_test()

In [ ]:
############## Conv1 ############
import numpy as np
int_conv1_weight_1 =  torch.tensor((model.conv1.weight.data[0][0] * 128), dtype = torch.int32)
int_conv1_weight_2 =  torch.tensor((model.conv1.weight.data[1][0] * 128), dtype = torch.int32)
int_conv1_weight_3 =  torch.tensor((model.conv1.weight.data[2][0] * 128), dtype = torch.int32)
int_conv1_weight_4 =  torch.tensor((model.conv1.weight.data[3][0] * 128), dtype = torch.int32)
#It is to prevent overflow of weights. If the weight value exceeds 127 (011111111), MSB becomes 1 when it is converted into a Binary of 8bit.
#However, in HW, this means negative. Therefore, the number exceeding 127 is fixed to 127.
for i in range(5):
    for j in range(5):
        if int_conv1_weight_1[i][j] > 127:
            int_conv1_weight_1[i][j] = 127
        if int_conv1_weight_2[i][j] > 127:
            int_conv1_weight_2[i][j] = 127
        if int_conv1_weight_3[i][j] > 127:
            int_conv1_weight_3[i][j] = 127
        if int_conv1_weight_4[i][j] > 127:
            int_conv1_weight_4[i][j] = 127
int_conv1_weight_stack = np.vstack((int_conv1_weight_1,int_conv1_weight_2,int_conv1_weight_3,int_conv1_weight_4))
print("Signed")
print(int_conv1_weight_stack)
print(int_conv1_weight_1)
print(int_conv1_weight_2)
print(int_conv1_weight_3)
print(int_conv1_weight_4)
np.savetxt('SW_conv1_weight.txt', int_conv1_weight_stack, fmt='%d',delimiter = " ")

# If the value of the weight is negative, then +256 based on 8 bits and extract it as the Hex value, HW accepts it as the correct negative value.
for i in range(5):
    for j in range(5):
        if int_conv1_weight_1[i][j] < 0:
            int_conv1_weight_1[i][j] += 256
        if int_conv1_weight_2[i][j] < 0:
            int_conv1_weight_2[i][j] += 256
        if int_conv1_weight_3[i][j] < 0:
            int_conv1_weight_3[i][j] += 256
        if int_conv1_weight_4[i][j] < 0:
            int_conv1_weight_4[i][j] += 256

int_conv1_weight_stack = np.vstack((int_conv1_weight_1,int_conv1_weight_2,int_conv1_weight_3,int_conv1_weight_4))
print ("Unsigned")
print(int_conv1_weight_stack)
print(int_conv1_weight_1)
print(int_conv1_weight_2)
print(int_conv1_weight_3)
print(int_conv1_weight_4)
np.savetxt('HW_conv1_weight.mem', int_conv1_weight_stack, fmt='%1.2X',delimiter = " ")

In [ ]:
############## Conv2 ############
import numpy as np
print(np.shape(model.conv2.weight))

int_conv2_weight_11 =  torch.tensor((model.conv2.weight.data[0][0]* 128), dtype = torch.int32)
int_conv2_weight_12 =  torch.tensor((model.conv2.weight.data[0][1]* 128), dtype = torch.int32)
int_conv2_weight_13 =  torch.tensor((model.conv2.weight.data[0][2]* 128), dtype = torch.int32)
int_conv2_weight_14 =  torch.tensor((model.conv2.weight.data[0][3]* 128), dtype = torch.int32)

int_conv2_weight_21 =  torch.tensor((model.conv2.weight.data[1][0] * 128), dtype = torch.int32)
int_conv2_weight_22 =  torch.tensor((model.conv2.weight.data[1][1] * 128), dtype = torch.int32)
int_conv2_weight_23 =  torch.tensor((model.conv2.weight.data[1][2] * 128), dtype = torch.int32)
int_conv2_weight_24 =  torch.tensor((model.conv2.weight.data[1][3] * 128), dtype = torch.int32)

int_conv2_weight_31 =  torch.tensor((model.conv2.weight.data[2][0] * 128), dtype = torch.int32)
int_conv2_weight_32 =  torch.tensor((model.conv2.weight.data[2][1] * 128), dtype = torch.int32)
int_conv2_weight_33 =  torch.tensor((model.conv2.weight.data[2][2] * 128), dtype = torch.int32)
int_conv2_weight_34 =  torch.tensor((model.conv2.weight.data[2][3] * 128), dtype = torch.int32)

int_conv2_weight_41 =  torch.tensor((model.conv2.weight.data[3][0] * 128), dtype = torch.int32)
int_conv2_weight_42 =  torch.tensor((model.conv2.weight.data[3][1] * 128), dtype = torch.int32)
int_conv2_weight_43 =  torch.tensor((model.conv2.weight.data[3][2] * 128), dtype = torch.int32)
int_conv2_weight_44 =  torch.tensor((model.conv2.weight.data[3][3] * 128), dtype = torch.int32)

int_conv2_weight_51 =  torch.tensor((model.conv2.weight.data[4][0] * 128), dtype = torch.int32)
int_conv2_weight_52 =  torch.tensor((model.conv2.weight.data[4][1] * 128), dtype = torch.int32)
int_conv2_weight_53 =  torch.tensor((model.conv2.weight.data[4][2] * 128), dtype = torch.int32)
int_conv2_weight_54 =  torch.tensor((model.conv2.weight.data[4][3] * 128), dtype = torch.int32)

int_conv2_weight_61 =  torch.tensor((model.conv2.weight.data[5][0] * 128), dtype = torch.int32)
int_conv2_weight_62 =  torch.tensor((model.conv2.weight.data[5][1] * 128), dtype = torch.int32)
int_conv2_weight_63 =  torch.tensor((model.conv2.weight.data[5][2] * 128), dtype = torch.int32)
int_conv2_weight_64 =  torch.tensor((model.conv2.weight.data[5][3] * 128), dtype = torch.int32)

int_conv2_weight_71 =  torch.tensor((model.conv2.weight.data[6][0] * 128), dtype = torch.int32)
int_conv2_weight_72 =  torch.tensor((model.conv2.weight.data[6][1] * 128), dtype = torch.int32)
int_conv2_weight_73 =  torch.tensor((model.conv2.weight.data[6][2] * 128), dtype = torch.int32)
int_conv2_weight_74 =  torch.tensor((model.conv2.weight.data[6][3] * 128), dtype = torch.int32)

int_conv2_weight_81 =  torch.tensor((model.conv2.weight.data[7][0] * 128), dtype = torch.int32)
int_conv2_weight_82 =  torch.tensor((model.conv2.weight.data[7][1] * 128), dtype = torch.int32)
int_conv2_weight_83 =  torch.tensor((model.conv2.weight.data[7][2] * 128), dtype = torch.int32)
int_conv2_weight_84 =  torch.tensor((model.conv2.weight.data[7][3] * 128), dtype = torch.int32)

int_conv2_weight_91 =  torch.tensor((model.conv2.weight.data[8][0] * 128), dtype = torch.int32)
int_conv2_weight_92 =  torch.tensor((model.conv2.weight.data[8][1] * 128), dtype = torch.int32)
int_conv2_weight_93 =  torch.tensor((model.conv2.weight.data[8][2] * 128), dtype = torch.int32)
int_conv2_weight_94 =  torch.tensor((model.conv2.weight.data[8][3] * 128), dtype = torch.int32)

int_conv2_weight_101 =  torch.tensor((model.conv2.weight.data[9][0] * 128), dtype = torch.int32)
int_conv2_weight_102 =  torch.tensor((model.conv2.weight.data[9][1] * 128), dtype = torch.int32)
int_conv2_weight_103 =  torch.tensor((model.conv2.weight.data[9][2] * 128), dtype = torch.int32)
int_conv2_weight_104 =  torch.tensor((model.conv2.weight.data[9][3] * 128), dtype = torch.int32)

int_conv2_weight_111 =  torch.tensor((model.conv2.weight.data[10][0] * 128), dtype = torch.int32)
int_conv2_weight_112 =  torch.tensor((model.conv2.weight.data[10][1] * 128), dtype = torch.int32)
int_conv2_weight_113 =  torch.tensor((model.conv2.weight.data[10][2] * 128), dtype = torch.int32)
int_conv2_weight_114 =  torch.tensor((model.conv2.weight.data[10][3] * 128), dtype = torch.int32)

int_conv2_weight_121 =  torch.tensor((model.conv2.weight.data[11][0] * 128), dtype = torch.int32)
int_conv2_weight_122 =  torch.tensor((model.conv2.weight.data[11][1] * 128), dtype = torch.int32)
int_conv2_weight_123 =  torch.tensor((model.conv2.weight.data[11][2] * 128), dtype = torch.int32)
int_conv2_weight_124 =  torch.tensor((model.conv2.weight.data[11][3] * 128), dtype = torch.int32)

for i in range(5):
    for j in range(5):
        if int_conv2_weight_11[i][j] > 127:
            int_conv2_weight_11[i][j] = 127
        if int_conv2_weight_12[i][j] > 127:
            int_conv2_weight_12[i][j] = 127
        if int_conv2_weight_13[i][j] > 127:
            int_conv2_weight_13[i][j] = 127
        if int_conv2_weight_14[i][j] > 127:
            int_conv2_weight_14[i][j] = 127
            
        if int_conv2_weight_21[i][j] > 127:
            int_conv2_weight_21[i][j] = 127
        if int_conv2_weight_22[i][j] > 127:
            int_conv2_weight_22[i][j] = 127
        if int_conv2_weight_23[i][j] > 127:
            int_conv2_weight_23[i][j] = 127
        if int_conv2_weight_24[i][j] > 127:
            int_conv2_weight_24[i][j] = 127   
             
        if int_conv2_weight_31[i][j] > 127:
            int_conv2_weight_31[i][j] = 127
        if int_conv2_weight_32[i][j] > 127:
            int_conv2_weight_32[i][j] = 127
        if int_conv2_weight_33[i][j] > 127:
            int_conv2_weight_33[i][j] = 127
        if int_conv2_weight_34[i][j] > 127:
            int_conv2_weight_34[i][j] = 127   
        
        if int_conv2_weight_41[i][j] > 127:
            int_conv2_weight_41[i][j] = 127
        if int_conv2_weight_42[i][j] > 127:
            int_conv2_weight_42[i][j] = 127
        if int_conv2_weight_43[i][j] > 127:
            int_conv2_weight_43[i][j] = 127
        if int_conv2_weight_44[i][j] > 127:
            int_conv2_weight_44[i][j] = 127
            
        if int_conv2_weight_51[i][j] > 127:
            int_conv2_weight_51[i][j] = 127
        if int_conv2_weight_52[i][j] > 127:
            int_conv2_weight_52[i][j] = 127
        if int_conv2_weight_53[i][j] > 127:
            int_conv2_weight_53[i][j] = 127
        if int_conv2_weight_54[i][j] > 127:
            int_conv2_weight_54[i][j] = 127   
             
        if int_conv2_weight_61[i][j] > 127:
            int_conv2_weight_61[i][j] = 127
        if int_conv2_weight_62[i][j] > 127:
            int_conv2_weight_62[i][j] = 127
        if int_conv2_weight_63[i][j] > 127:
            int_conv2_weight_63[i][j] = 127
        if int_conv2_weight_64[i][j] > 127:
            int_conv2_weight_64[i][j] = 127
        
        if int_conv2_weight_71[i][j] > 127:
            int_conv2_weight_71[i][j] = 127
        if int_conv2_weight_72[i][j] > 127:
            int_conv2_weight_72[i][j] = 127
        if int_conv2_weight_73[i][j] > 127:
            int_conv2_weight_73[i][j] = 127
        if int_conv2_weight_74[i][j] > 127:
            int_conv2_weight_74[i][j] = 127
            
        if int_conv2_weight_81[i][j] > 127:
            int_conv2_weight_81[i][j] = 127
        if int_conv2_weight_82[i][j] > 127:
            int_conv2_weight_82[i][j] = 127
        if int_conv2_weight_83[i][j] > 127:
            int_conv2_weight_83[i][j] = 127
        if int_conv2_weight_84[i][j] > 127:
            int_conv2_weight_84[i][j] = 127   
             
        if int_conv2_weight_91[i][j] > 127:
            int_conv2_weight_91[i][j] = 127
        if int_conv2_weight_92[i][j] > 127:
            int_conv2_weight_92[i][j] = 127
        if int_conv2_weight_93[i][j] > 127:
            int_conv2_weight_93[i][j] = 127
        if int_conv2_weight_94[i][j] > 127:
            int_conv2_weight_94[i][j] = 127  
        
        if int_conv2_weight_101[i][j] > 127:
            int_conv2_weight_101[i][j] = 127
        if int_conv2_weight_102[i][j] > 127:
            int_conv2_weight_102[i][j] = 127
        if int_conv2_weight_103[i][j] > 127:
            int_conv2_weight_103[i][j] = 127
        if int_conv2_weight_104[i][j] > 127:
            int_conv2_weight_104[i][j] = 127
            
        if int_conv2_weight_111[i][j] > 127:
            int_conv2_weight_111[i][j] = 127
        if int_conv2_weight_112[i][j] > 127:
            int_conv2_weight_112[i][j] = 127
        if int_conv2_weight_113[i][j] > 127:
            int_conv2_weight_113[i][j] = 127
        if int_conv2_weight_114[i][j] > 127:
            int_conv2_weight_114[i][j] = 127 
             
        if int_conv2_weight_121[i][j] > 127:
            int_conv2_weight_121[i][j] = 127
        if int_conv2_weight_122[i][j] > 127:
            int_conv2_weight_122[i][j] = 127
        if int_conv2_weight_123[i][j] > 127:
            int_conv2_weight_123[i][j] = 127
        if int_conv2_weight_124[i][j] > 127:
            int_conv2_weight_124[i][j] = 127
            
int_conv2_weight_stack = np.vstack((int_conv2_weight_11,int_conv2_weight_21,int_conv2_weight_31,int_conv2_weight_41,int_conv2_weight_51,int_conv2_weight_61,
                                        int_conv2_weight_71,int_conv2_weight_81,int_conv2_weight_91,int_conv2_weight_101,int_conv2_weight_111,int_conv2_weight_121,
                                        int_conv2_weight_12,int_conv2_weight_22,int_conv2_weight_32,int_conv2_weight_42,int_conv2_weight_52,int_conv2_weight_62,
                                        int_conv2_weight_72,int_conv2_weight_82,int_conv2_weight_92,int_conv2_weight_102,int_conv2_weight_112,int_conv2_weight_122,
                                        int_conv2_weight_13,int_conv2_weight_23,int_conv2_weight_33,int_conv2_weight_43,int_conv2_weight_53,int_conv2_weight_63,
                                        int_conv2_weight_73,int_conv2_weight_83,int_conv2_weight_93,int_conv2_weight_103,int_conv2_weight_113,int_conv2_weight_123,
                                        int_conv2_weight_14,int_conv2_weight_24,int_conv2_weight_34,int_conv2_weight_44,int_conv2_weight_54,int_conv2_weight_64,
                                        int_conv2_weight_74,int_conv2_weight_84,int_conv2_weight_94,int_conv2_weight_104,int_conv2_weight_114,int_conv2_weight_124))

print ("Signed")
print(int_conv2_weight_stack, '\n')
np.savetxt('SW_conv2_weight.txt', int_conv2_weight_stack, fmt='%d',delimiter = " ")

# signed int => unsigned int
for i in range(5):
    for j in range(5):
        if int_conv2_weight_11[i][j] < 0:
            int_conv2_weight_11[i][j] += 256
        if int_conv2_weight_12[i][j] < 0:
            int_conv2_weight_12[i][j] += 256
        if int_conv2_weight_13[i][j] < 0:
            int_conv2_weight_13[i][j] += 256
        if int_conv2_weight_14[i][j] < 0:
            int_conv2_weight_14[i][j] += 256
            
        if int_conv2_weight_21[i][j] < 0:
            int_conv2_weight_21[i][j] += 256
        if int_conv2_weight_22[i][j] < 0:
            int_conv2_weight_22[i][j] += 256
        if int_conv2_weight_23[i][j] < 0:
            int_conv2_weight_23[i][j] += 256
        if int_conv2_weight_24[i][j] < 0:
            int_conv2_weight_24[i][j] += 256   
             
        if int_conv2_weight_31[i][j] < 0:
            int_conv2_weight_31[i][j] += 256
        if int_conv2_weight_32[i][j] < 0:
            int_conv2_weight_32[i][j] += 256
        if int_conv2_weight_33[i][j] < 0:
            int_conv2_weight_33[i][j] += 256
        if int_conv2_weight_34[i][j] < 0:
            int_conv2_weight_34[i][j] += 256   
        
        if int_conv2_weight_41[i][j] < 0:
            int_conv2_weight_41[i][j] += 256
        if int_conv2_weight_42[i][j] < 0:
            int_conv2_weight_42[i][j] += 256
        if int_conv2_weight_43[i][j] < 0:
            int_conv2_weight_43[i][j] += 256
        if int_conv2_weight_44[i][j] < 0:
            int_conv2_weight_44[i][j] += 256
            
        if int_conv2_weight_51[i][j] < 0:
            int_conv2_weight_51[i][j] += 256
        if int_conv2_weight_52[i][j] < 0:
            int_conv2_weight_52[i][j] += 256
        if int_conv2_weight_53[i][j] < 0:
            int_conv2_weight_53[i][j] += 256
        if int_conv2_weight_54[i][j] < 0:
            int_conv2_weight_54[i][j] += 256   
             
        if int_conv2_weight_61[i][j] < 0:
            int_conv2_weight_61[i][j] += 256
        if int_conv2_weight_62[i][j] < 0:
            int_conv2_weight_62[i][j] += 256
        if int_conv2_weight_63[i][j] < 0:
            int_conv2_weight_63[i][j] += 256
        if int_conv2_weight_64[i][j] < 0:
            int_conv2_weight_64[i][j] += 256
        
        if int_conv2_weight_71[i][j] < 0:
            int_conv2_weight_71[i][j] += 256
        if int_conv2_weight_72[i][j] < 0:
            int_conv2_weight_72[i][j] += 256
        if int_conv2_weight_73[i][j] < 0:
            int_conv2_weight_73[i][j] += 256
        if int_conv2_weight_74[i][j] < 0:
            int_conv2_weight_74[i][j] += 256
            
        if int_conv2_weight_81[i][j] < 0:
            int_conv2_weight_81[i][j] += 256
        if int_conv2_weight_82[i][j] < 0:
            int_conv2_weight_82[i][j] += 256
        if int_conv2_weight_83[i][j] < 0:
            int_conv2_weight_83[i][j] += 256
        if int_conv2_weight_84[i][j] < 0:
            int_conv2_weight_84[i][j] += 256   
             
        if int_conv2_weight_91[i][j] < 0:
            int_conv2_weight_91[i][j] += 256
        if int_conv2_weight_92[i][j] < 0:
            int_conv2_weight_92[i][j] += 256
        if int_conv2_weight_93[i][j] < 0:
            int_conv2_weight_93[i][j] += 256
        if int_conv2_weight_94[i][j] < 0:
            int_conv2_weight_94[i][j] += 256   
        
        if int_conv2_weight_101[i][j] < 0:
            int_conv2_weight_101[i][j] += 256
        if int_conv2_weight_102[i][j] < 0:
            int_conv2_weight_102[i][j] += 256
        if int_conv2_weight_103[i][j] < 0:
            int_conv2_weight_103[i][j] += 256
        if int_conv2_weight_104[i][j] < 0:
            int_conv2_weight_104[i][j] += 256
            
        if int_conv2_weight_111[i][j] < 0:
            int_conv2_weight_111[i][j] += 256
        if int_conv2_weight_112[i][j] < 0:
            int_conv2_weight_112[i][j] += 256
        if int_conv2_weight_113[i][j] < 0:
            int_conv2_weight_113[i][j] += 256
        if int_conv2_weight_114[i][j] < 0:
            int_conv2_weight_114[i][j] += 256   
             
        if int_conv2_weight_121[i][j] < 0:
            int_conv2_weight_121[i][j] += 256
        if int_conv2_weight_122[i][j] < 0:
            int_conv2_weight_122[i][j] += 256
        if int_conv2_weight_123[i][j] < 0:
            int_conv2_weight_123[i][j] += 256
        if int_conv2_weight_124[i][j] < 0:
            int_conv2_weight_124[i][j] += 256   
        

int_conv2_weight_stack_ch1 = np.vstack((int_conv2_weight_11,int_conv2_weight_21,int_conv2_weight_31,int_conv2_weight_41,int_conv2_weight_51,int_conv2_weight_61,
                                        int_conv2_weight_71,int_conv2_weight_81,int_conv2_weight_91,int_conv2_weight_101,int_conv2_weight_111,int_conv2_weight_121))
int_conv2_weight_stack_ch2 = np.vstack((int_conv2_weight_12,int_conv2_weight_22,int_conv2_weight_32,int_conv2_weight_42,int_conv2_weight_52,int_conv2_weight_62,
                                        int_conv2_weight_72,int_conv2_weight_82,int_conv2_weight_92,int_conv2_weight_102,int_conv2_weight_112,int_conv2_weight_122))
int_conv2_weight_stack_ch3 = np.vstack((int_conv2_weight_13,int_conv2_weight_23,int_conv2_weight_33,int_conv2_weight_43,int_conv2_weight_53,int_conv2_weight_63,
                                        int_conv2_weight_73,int_conv2_weight_83,int_conv2_weight_93,int_conv2_weight_103,int_conv2_weight_113,int_conv2_weight_123))
int_conv2_weight_stack_ch4 = np.vstack((int_conv2_weight_14,int_conv2_weight_24,int_conv2_weight_34,int_conv2_weight_44,int_conv2_weight_54,int_conv2_weight_64,
                                        int_conv2_weight_74,int_conv2_weight_84,int_conv2_weight_94,int_conv2_weight_104,int_conv2_weight_114,int_conv2_weight_124))

print ("Unsigned")
print(int_conv2_weight_stack_ch1, '\n')
print(int_conv2_weight_stack_ch2, '\n')
print(int_conv2_weight_stack_ch3, '\n')
print(int_conv2_weight_stack_ch4, '\n')

np.savetxt('HW_conv2_weight_ch1.mem', int_conv2_weight_stack_ch1, fmt='%1.2X',delimiter = " ")
np.savetxt('HW_conv2_weight_ch2.mem', int_conv2_weight_stack_ch2, fmt='%1.2X',delimiter = " ")
np.savetxt('HW_conv2_weight_ch3.mem', int_conv2_weight_stack_ch3, fmt='%1.2X',delimiter = " ")
np.savetxt('HW_conv2_weight_ch4.mem', int_conv2_weight_stack_ch4, fmt='%1.2X',delimiter = " ")



In [ ]:
############## FC Layer ############

print(np.shape(model.fc_1.weight))
print((model.fc_1.weight * 128).int())

print(np.shape(model.fc_1.bias))
print((model.fc_1.bias * 128 * 256).int())

int_fc_weight = (model.fc_1.weight * 128).int()
int_fc_bias = (model.fc_1.bias * 128 * 256).int() #16bit bias
for i in range(10):
    for j in range(192):
        if int_fc_weight[i][j] > 127 :
            int_fc_weight[i][j] = 127
            
np.savetxt('SW_fc_weight.txt', int_fc_weight, fmt='%d',delimiter = " ")
np.savetxt('SW_fc_bias.txt', int_fc_bias, fmt='%d',delimiter = " ")
# signed int => unsigned int
for i in range(10):
    for j in range(192):
        if int_fc_weight[i][j] < 0 :
            int_fc_weight[i][j] += 256
    if int_fc_bias[i] < 0 :
        int_fc_bias[i] += 256*256 #16bit bias
        
print(int_fc_weight)
print(int_fc_bias)

np.savetxt('HW_fc_weight.mem', int_fc_weight, fmt='%1.2X',delimiter = " ")
np.savetxt('HW_fc_bias.mem', int_fc_bias, fmt='%1.2X',delimiter = " ")